# Limpiar dataset

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv("../data/DetalleFacturas.csv", encoding="latin-1")

In [27]:
import re

invalid = [
    # Tarifas y cargos
    'tarifa', 'cargo', 'contribución', 'gubernamental',
    
    # Servicios
    'hospedaje', 'lodging', 'consumo de alimentos', 'consumo del día',
    'alimentos y bebidas', 'servicios de cafetería', 'restaurante',
    
    # Códigos/Referencias
    'pnr:', 'folio', 'ticket', 'según número', 'seg�n',
    
    # Descriptores genéricos
    'consumo', 'no. mesa', 'cant pcio', 'total', 'subtotal',
    'descuento', 'receipt', 'tpv adyen',
    
    # Servicios específicos
    'corte en guillotina', 'vinil en coroplast',
]

def is_valid_product(description):
    if pd.isna(description):
        return False
    
    desc_lower = str(description).lower()
    desc_str = str(description)

    # Filtrar patrones inválidos
    for pattern in invalid:
        if pattern in desc_lower:
            return False
    
    # Filtrar códigos alfanuméricos raros (patrón: 2 letras + 10 dígitos + nombre)
    # Ejemplos: ND3335711602, SC3115547608, OD3104793220, PC3617130905
    if re.match(r'^[A-Z]{2}\d{10}', desc_str):
        return False
        
    # Filtrar descripciones muy cortas
    if len(desc_lower.strip()) < 3:
        return False
    
    return True

In [28]:
df['is_valid'] = df['vcDescripcion'].apply(is_valid_product)
df_clean = df[df['is_valid'] == True].copy().reset_index(drop=True)  # reset_index para evitar warnings

print(f"Original: {len(df)} productos")
print(f"Después de limpieza: {len(df_clean)} productos")

Original: 1000 productos
Después de limpieza: 702 productos


In [29]:
# Agregar columna 'type' usando suggest_category con umbral

# Cargar categorías de convertcsv.csv
df_categories = pd.read_csv("../data/convertcsv.csv", encoding="latin-1")

valid_categories = df_categories['type'].dropna().unique()
valid_categories = [cat for cat in valid_categories if cat and cat != 'discount']

print(f"Categorías disponibles: {valid_categories}")

# Cargar modelo de embeddings
import sys
sys.path.append('..')

from services.embeddings import generate_embeddings, cosine_similarity

# Generar embeddings de las categorías
category_embeddings = generate_embeddings(list(valid_categories))

print(f"\nGenerando embeddings para {len(df_clean)} productos...")

# Umbral de confianza (si el score es menor, asignar 'other')
CONFIDENCE_THRESHOLD = 0.15

# Función para sugerir categoría con umbral
def suggest_category_for_product(product_name):
    if pd.isna(product_name) or not product_name:
        return "other"
    
    # Generar embedding del producto
    product_emb = generate_embeddings(str(product_name))[0]
    
    # Calcular similitud con cada categoría
    scores = [cosine_similarity(product_emb, cat_emb) for cat_emb in category_embeddings]
    
    # Obtener mejor score y categoría
    max_score = max(scores)
    best_idx = scores.index(max_score)
    
    # Si el score es muy bajo, categoría es incierta -> 'other'
    if max_score < CONFIDENCE_THRESHOLD:
        return "other"
    
    return valid_categories[best_idx]

# Aplicar suggest_category a todos los productos
df_clean.loc[:, 'type'] = df_clean['vcDescripcion'].apply(suggest_category_for_product)

print(f"\nDistribución de categorías:")
print(df_clean['type'].value_counts())

# Ver productos categorizados como 'other'
other_products = df_clean[df_clean['type'] == 'other']
if len(other_products) > 0:
    print(f"\n{len(other_products)} productos categorizados como 'other' (baja confianza):")
    print(other_products[['vcDescripcion', 'type']].head(10))

Categorías disponibles: ['product', 'food', 'alcohol', 'tobacco', 'fuel']

Generando embeddings para 702 productos...

Distribución de categorías:
type
product    228
food       176
other      108
tobacco     85
fuel        74
alcohol     31
Name: count, dtype: int64

108 productos categorizados como 'other' (baja confianza):
                                         vcDescripcion   type
9          PERFIL TUBULAR RECTANGULAR 2"X2" C-14 C/6MT  other
17   SERVICIO DE TRANSPORTE DE PRIMERA PLUS CYA-MOR...  other
24   SANDWICH KIRLUNCH TRIPLE INTEGRAL JAMON PAVO 180G  other
27                                    MVPIZZA and STIX  other
42                      TIMBER ROUND ANTHRACITE 160 CM  other
49                  KOTEX NOTCURNA ULTRADELGADA CA 10S  other
54               GALLETAS INTEGRALES ARANDANO BITZ 80G  other
98                   GALLETAS INTEGRALES PINA BITZ 80G  other
103            LECHE PAST LALA DESLAC 2% 1.80 LT PL PG  other
122                            R20OZ CAPUCHINO REGUL

In [30]:
# Ver ejemplos de productos con su categoría inferida
print("\nEjemplos de productos con categoría:")
print(df_clean[['vcDescripcion', 'type']].head(20))


Ejemplos de productos con categoría:
                                        vcDescripcion     type
0                            Caramel Frappuccino Alto  alcohol
1                                  leche deslactosada  tobacco
2                           CHOCOLATINES CHAROLA 98GR     food
3                                  DORITOS NACHO 61GR     food
4   REFRESCO COCACOLA NO RETORNABLE 450 ML BOTELLA...  tobacco
5                                COCA COLA  PET 450ML  alcohol
6                             Pan de Elote 1pz Bakery     food
7     Energizer Pila Recargable AA Paquete de 2 Pilas  product
8     Energizer Pila Recargable AA Paquete de 2 Pilas  product
9         PERFIL TUBULAR RECTANGULAR 2"X2" C-14 C/6MT    other
10  SOLDADURA INFRA/FLEXARC 6013 1/8" PUNTA NARANJ...     food
11             SOLDADURA INFRA/CHAMPION 6013 3/32" KG     food
12                LENTES MILWAUKEE GRIS ANTIRAYADURAS  alcohol
13                      VIDRIO PARA CARETA 12 SOMBRAS  tobacco
14               

In [31]:
# Guardar CSV limpio con columna 'type'
df_clean.to_csv("../data/DetalleFacturas_clean.csv", index=False, encoding="latin-1")
print("\nArchivo guardado: DetalleFacturas_clean.csv con columna 'type'")


Archivo guardado: DetalleFacturas_clean.csv con columna 'type'
